In [ ]:
import pandas as pd
import numpy as np
import xgboost as xgb
import time
import tensorflow as tf
import tensorflow.keras as keras
from sklearn.preprocessing import StandardScaler
import math
import sklearn
from sklearn.ensemble import RandomForestClassifier
import warnings
import eli5
import datatable as dt
import matplotlib.pyplot as plt
import os
import csv
import cv2
from tensorflow.keras.preprocessing import image
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
import tensorflow_addons as tfa
from numpy import copy


In [ ]:
for (root, dirs, files) in os.walk('../input/hpa-single-cell-image-classification/train'):
    #image_dir=files
    i=len(files)

In [ ]:
df=pd.read_csv('../input/hpa-single-cell-image-classification/train.csv')
df.count()

In [ ]:
train_numpy=df.to_numpy()
print(train_numpy.shape)
print(train_numpy[0][0])
print(train_numpy[0][1])

In [ ]:

train_numpy_red=copy(train_numpy)
print(train_numpy_red.shape)


for i in range(len(train_numpy)):
    train_numpy_red[i]=np.array([(train_numpy[i][0]+'_red.png'), train_numpy[i][1]])
#print(train_numpy_red.shape)
#print(train_numpy_red[0])
#print(train_numpy_red[1])

#print(train_numpy[0])
#print(train_numpy[1])

train_numpy_blue=copy(train_numpy)
#print(train_numpy_blue.shape)


for i in range(len(train_numpy)):
    train_numpy_blue[i]=np.array([(train_numpy[i][0]+'_blue.png'), train_numpy[i][1]])
#print(train_numpy_blue.shape)
#print(train_numpy_blue[0])
#print(train_numpy_blue[1])

#print(train_numpy[0])
#print(train_numpy[1])


train_numpy_green=copy(train_numpy)
#print(train_numpy_blue.shape)


for i in range(len(train_numpy)):
    train_numpy_green[i]=np.array([(train_numpy[i][0]+'_green.png'), train_numpy[i][1]])
#print(train_numpy_green.shape)
#print(train_numpy_green[0])
#print(train_numpy_green[1])

#print(train_numpy[0])
#print(train_numpy[1])

train_numpy_yellow=copy(train_numpy)
#print(train_numpy_yellow.shape)


for i in range(len(train_numpy)):
    train_numpy_yellow[i]=np.array([(train_numpy[i][0]+'_yellow.png'), train_numpy[i][1]])
#print(train_numpy_yellow.shape)
#print(train_numpy_yellow[0])
#print(train_numpy_yellow[1])

#print(train_numpy[0])
#print(train_numpy[1])


In [ ]:
train_numpy_new=copy(train_numpy_blue)
train_numpy_new=np.append(train_numpy_new, train_numpy_yellow, axis=0)
train_numpy_new=np.append(train_numpy_new, train_numpy_green, axis=0)
train_numpy_new=np.append(train_numpy_new, train_numpy_red, axis=0)
print(train_numpy_new.shape)
#print(train_numpy_new[0])
#print(train_numpy_new[21805])
#print(train_numpy_new[21806])
#print(train_numpy_new[43611])
#print(train_numpy_new[43612])
#print(train_numpy_new[65417])
#print(train_numpy_new[65418])
#print(train_numpy_new[87223])


train_df_new=pd.DataFrame(data=train_numpy_new, columns=['id', 'labels'])
#print(train_df_new.count())
#print(train_df_new.head(10))
#print(train_df_new.tail(10))

train_df_new['labels'] = train_df_new['labels'].apply(lambda string: string.split('|'))
#print(train_df_new.count())
#print(train_df_new.head(10))
#print(train_df_new.tail(10))


In [ ]:
prob_threshold=0.4
img_shape=(256, 256, 3)
n_ohe=19
n_h, n_w, n_c=img_shape
batch_size=256
validation_split=0.2

alpha_lr1=0.0005
alpha_lr2=0.0001

epochs_num1=1
epochs_num2=1

#Second iteration of model run
trainable_mid=11
non_trainable_end=22

In [ ]:
datagen = keras.preprocessing.image.ImageDataGenerator(rescale=1/255.0,
                                                        preprocessing_function=None,
                                                        data_format=None,
                                                        validation_split=validation_split)

In [ ]:
start_time=time.time()

train_data = datagen.flow_from_dataframe(
    train_df_new,
    directory='../input/hpa-single-cell-image-classification/train',
    x_col="id",
    y_col= 'labels',
    color_mode="rgb",
    target_size = (n_h, n_w),
    class_mode="categorical",
    batch_size=batch_size,
    shuffle=True,
    seed=40,
    subset='training')

end_time=time.time()

print('Time taken is ', (end_time-start_time)/60, ' mins')

In [ ]:
validation_data = datagen.flow_from_dataframe(
    train_df_new,
    directory='../input/hpa-single-cell-image-classification/train',
    x_col="id",
    y_col= 'labels',
    color_mode="rgb",
    target_size = (n_h, n_w),
    class_mode="categorical",
    batch_size=batch_size,
    shuffle=True,
    seed=40,
    subset='validation')

In [ ]:
weights_path = '../input/keras-pretrained-models/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5'
VGG16_MODEL = keras.applications.VGG16(weights=weights_path ,include_top=False, input_shape=(n_h, n_w, n_c))
x=VGG16_MODEL.output
x=GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
prediction=Dense(n_ohe, activation='sigmoid')(x)

In [ ]:
for layer in VGG16_MODEL.layers:
    layer.trainable=False

In [ ]:
model=Model(inputs=VGG16_MODEL.input, outputs=prediction)
f1_score = tfa.metrics.F1Score(num_classes=n_ohe, threshold=prob_threshold, average='micro')
model.compile(tf.keras.optimizers.Adam(learning_rate=alpha_lr1) , loss='binary_crossentropy', metrics=[f1_score])

In [ ]:
start_time=time.time()

model.fit(train_data, epochs=epochs_num1, validation_data=validation_data)

end_time=time.time()

print('Time taken is ', (end_time-start_time)/60., ' mins'  )

In [ ]:
i=0
import cv2

subm_array=np.array([])

for dirname, _, filenames in os.walk('../input/hpa-single-cell-image-classification/test'):
    for filename in filenames:
        #filename_arr.append(filename)
        img=cv2.imread(os.path.join('../input/hpa-single-cell-image-classification/test/', filename))
        img_shape=list(img.shape)
        
        subm = np.array( [[filename, img_shape[0], img_shape[1],  'a']])
        #print(subm.shape)
        #subm_array= np.append(subm_array, subm)
        if i == 0:
            subm_array=subm
        else:
            subm_array= np.append(subm_array, subm, axis=0)
        i+=1

#print(subm_array)
print(subm_array.shape)
test_data_size=subm_array.shape[0]

test = pd.DataFrame(subm_array, columns = ['id', 'height', 'width', 'labels'])

datagen = keras.preprocessing.image.ImageDataGenerator(rescale=1/255.0,
                                                        preprocessing_function=None,
                                                        data_format=None,
                                                    )




test_data = datagen.flow_from_dataframe(
    test,
    directory='../input/hpa-single-cell-image-classification/test/',
    x_col="id",
    y_col= 'labels',
    color_mode="rgb",
    target_size = (n_h, n_w),
    class_mode="raw",
    batch_size=batch_size,
    shuffle=False,
    seed=40
)

In [ ]:
predict_test=model.predict(test_data)
print(type(predict_test))